In [ ]:
%pip install roboflow
%pip install opencv-python-headless
%pip install torch torchvision torchaudio
%pip install pyyaml
%pip install tqdm
%pip install roboflow

In [2]:
# Dataset 1

# from roboflow import Roboflow
# rf = Roboflow(api_key="xx")
# project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
# version = project.version(4)
# dataset = version.download("yolov5")

In [3]:
# # Dataset 2

# from roboflow import Roboflow
# rf = Roboflow(api_key="xx")
# project = rf.workspace("augmented-startups").project("vehicle-registration-plates-trudk")
# version = project.version(1)
# dataset = version.download("yolov5")


In [4]:
# from ultralytics import YOLO

In [5]:
# yolo=YOLO("weights/yolov5su.pt")
# yolo.train(data="License-Plate-Recognition-4/data.yaml", epochs=30, batch=-1)
# valid_results=yolo.val()
# print("Validation Results")
# print(valid_results)

In [6]:
# yolo=YOLO("runs_1/detect/train7/weights/last.pt")
# yolo.train(data="Vehicle-Registration-Plates-1/data.yaml", epochs=30, batch=-1)
# valid_results=yolo.val()
# print("Validation Results")
# print(valid_results)

In [7]:
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def draw_bbox(image, results):
	classes = results.names
	if isinstance(image, str):
		image = Image.open(image)
		image = np.array(image)

	for i in range(len(results.boxes)):
		box = results.boxes[i]
		tensor = box.xyxy[0]
		if box.cls in [2,3,5,7]:
			classname = classes[int(box.cls)]
			x1 = int(tensor[0].item())
			y1 = int(tensor[1].item())
			x2 = int(tensor[2].item())
			y2 = int(tensor[3].item())
			cv2.rectangle(image,(x1,y1),(x2,y2),(255,0,255),3)
			# cv2.putText(image, classname, (int(x1) + 5, int(y1) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,255), 2)
	return image

def predict_image(model, image):
	results = model.predict(image)[0]
	image = draw_bbox(image, results)
	return image

def process_video(model, input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {input_path}")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to PIL Image for prediction
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        # Predict and draw bounding boxes
        results = model.predict(image)[0]
        frame = draw_bbox(np.array(image), results)
        
        # Write the frame into the output video
        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    # Release everything if job is finished
    cap.release()
    out.release()
    print(f"Processed video saved as {output_path}")

def forward(img):
        out_img = np.copy(img)
        img = cv2.resize(img, (out_img.shape[1], out_img.shape[0]))
        return out_img


def process_stream(model, input_stream):
    cap = cv2.VideoCapture(input_stream)
    frame_count = 0
    while(cap.isOpened()):
        frame_count += 1
        ret, frame = cap.read()
        if ret:
            out_frame = forward(frame)
            out_frame = cv2.resize(out_frame, (640, 480))
            if frame_count % 1 == 0:
                out_frame = predict_image(model, out_frame)
            cv2.imshow('Number Plate Detection', out_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

In [8]:
from ultralytics import YOLO
model = YOLO("weights/yolov8x.pt")
# predict_image(model, "License-Plate-Recognition-4/test/images/xemayBigPlate48_jpg.rf.f1383185ead4cbc0663c837ee7c884ed.jpg")
path = "1.mp4"
process_video(model, "Video/"+path, "Video/Predict/"+path)
path = "2.mp4"
process_video(model, "Video/"+path, "Video/Predict/"+path)
# process_stream(model, "License-Plate-Recognition-4/Video/"+path)


In [ ]:
# model = YOLO("runs_1/detect/train7/weights/best.pt")
# predict_image(model, "License-Plate-Recognition-4/test/images/xemayBigPlate48_jpg.rf.f1383185ead4cbc0663c837ee7c884ed.jpg")
# path = "2.mp4"
# # process_video(model, "License-Plate-Recognition-4/Video/"+path, "License-Plate-Recognition-4/Video/Predict/"+path)
# process_stream(model, "License-Plate-Recognition-4/Video/"+path)


In [ ]:
# model = YOLO("runs/detect/train/weights/best.pt")
# predict_image(model, "Vehicle-Registration-Plates-1/test/images/00a7d31c6cc6b7f3_jpg.rf.805219027ce99b4baa1deaba3679211b.jpg")
# path = "2.mp4"
# # process_video(model, "Vehicle-Registration-Plates-1/Video"+path, "License-Plate-Recognition-4/Video/Predict/"+path)
# process_stream(model, "Vehicle-Registration-Plates-1/Video/"+path)
